### 기상청 날씨 정보 조회_API
- 공공데이터 포털의 기상청 날씨조회 [활용요청] 후 사용
- https://www.data.go.kr/iim/api/selectAPIAcountView.do
- 참고문서를 이용한 API 사용법 확인

In [1]:
# 설명을 위해 변수 한글 사용
def urlprn(시작일, 종료일, 지점코드, 행갯수 = 10):
    인증키 = "rx1J4yPTJLMypgEsmw%2BuyunTeZzwwUbRPrwPUkkYNxyUDpxomPQu1Hjz8vXE%2BzLfrzvFQdKPXqp%2FvonN%2Bc77iA%3D%3D"
    url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey="
    url = url + 인증키
    url = url + "&pageNo=1&dataCd=ASOS&dateCd=DAY&dataType=XML"
    url = url + "&numOfRows=" + str(행갯수)
    url = url + "&startDt=" + str(시작일) + "&endDt=" + str(종료일)
    url = url + "&stnIds=" + str(지점코드)

    #print(url)
    
    # url을 이용해 데이터 요청
    soup_html = requests.get(url)
    if soup_html.status_code != 200:
        exit("데이터를 받아오지 못함")

    soup = bs(soup_html.text, "html.parser")

    return soup

In [2]:
import requests
from bs4 import BeautifulSoup as bs

In [3]:
행갯수 = 10
시작일 = 20220101
종료일 = 20220131
지점코드 = 108

In [4]:
soup = urlprn(시작일, 종료일, 지점코드)
행갯수 = soup.find("totalcount").get_text()
soup = urlprn(시작일, 종료일, 지점코드, 행갯수)

In [5]:
items = soup.find_all("item")
print(len(items))
print(items[0])
print(items[0].find("stnnm").get_text())

31
<item><stnid>108</stnid><stnnm>서울</stnnm><tm>2022-01-01</tm><avgta>-4.3</avgta><minta>-10.2</minta><mintahrmt>710</mintahrmt><maxta>2.3</maxta><maxtahrmt>1544</maxtahrmt><mi10maxrn></mi10maxrn><mi10maxrnhrmt></mi10maxrnhrmt><hr1maxrn></hr1maxrn><hr1maxrnhrmt></hr1maxrnhrmt><sumrndur></sumrndur><sumrn></sumrn><maxinsws>4.5</maxinsws><maxinswswd>70</maxinswswd><maxinswshrmt>923</maxinswshrmt><maxws>2.8</maxws><maxwswd>20</maxwswd><maxwshrmt>819</maxwshrmt><avgws>1.5</avgws><hr24sumrws>1335</hr24sumrws><maxwd>50</maxwd><avgtd>-14.4</avgtd><minrhm>31</minrhm><minrhmhrmt>1329</minrhmhrmt><avgrhm>46.3</avgrhm><avgpv>2.1</avgpv><avgpa>1019.8</avgpa><maxps>1034.0</maxps><maxpshrmt>247</maxpshrmt><minps>1027.3</minps><minpshrmt>2351</minpshrmt><avgps>1030.9</avgps><ssdur>9.6</ssdur><sumsshr>9.0</sumsshr><hr1maxicsrhrmt>1200</hr1maxicsrhrmt><hr1maxicsr>1.82</hr1maxicsr><sumgsr>10.39</sumgsr><ddmefs></ddmefs><ddmefshrmt></ddmefshrmt><ddmes></ddmes><ddmeshrmt></ddmeshrmt><sumdpthfhsc></sumdpthf

In [6]:
#### 필요 데이터 1개 가져오기

In [7]:
print(items[0].find("stnnm").get_text())
print(items[0].find("tm").get_text())
print(items[0].find("avgta").get_text())
print(items[0].find("minta").get_text())
print(items[0].find("maxta").get_text())
print(items[0].find("sumrn").get_text())
print(items[0].find("maxws").get_text())
print(items[0].find("avgws").get_text())

서울
2022-01-01
-4.3
-10.2
2.3

2.8
1.5


#### 조회 데이터 전체를 가져와 DataFrame으로 변경하기
1. items를 for문으로 데이터 갯수 만큼 반복 실행
2. 읽어온 item에서 필요한 데이터 추출 후 변수에 저장
3. 변수를 리스트에 저장
4. pandas 모듈을 이용해 DataFrame 형식으로 변경

In [8]:
itemtxt = []
 
for item in items:
    stnnm = item.find("stnnm").get_text()
    tm = item.find("tm").get_text()
    avgta = item.find("avgta").get_text()
    minta = item.find("minta").get_text()
    maxta = item.find("maxta").get_text()
    sumrn = item.find("sumrn").get_text()
    maxws = item.find("maxws").get_text()
    avgws = item.find("avgws").get_text()
    
    itemtxt.append({"지점명":stnnm , 
                 "일시":tm, 
                 "평균기온":avgta,
                 "최저기온":minta,
                 "최고기온":maxta,
                 "일일강수량":sumrn,
                 "최대풍속":maxws,
                 "평균풍속":avgws})
    
import pandas as pd

df1 = pd.DataFrame(itemtxt)
df1


,지점명,일시,평균기온,최저기온,최고기온,일일강수량,최대풍속,평균풍속
0,서울,2022-01-01,-4.3,-10.2,2.3,,2.8,1.5
1,서울,2022-01-02,-1.3,-5.2,3.0,0.3,4.6,2.3
2,서울,2022-01-03,-1.9,-8.0,2.5,0.0,4.1,1.8
3,서울,2022-01-04,-2.5,-5.6,1.0,0.0,4.2,2.4
4,서울,2022-01-05,-2.8,-7.8,1.9,,2.9,1.7
5,서울,2022-01-06,-2.2,-5.9,3.3,,4.6,2.3
6,서울,2022-01-07,-1.6,-5.9,4.5,,3.0,1.5
7,서울,2022-01-08,0.3,-5.0,3.7,0.0,3.5,1.7
8,서울,2022-01-09,1.3,-1.2,5.6,,2.8,1.7
9,서울,2022-01-10,0.5,-3.4,4.4,0.9,3.4,2.0


### Json 형식으로 가져오기

In [9]:
# 설명을 위해 변수 한글 사용
def urljson(시작일, 종료일, 지점코드, 행갯수 = 10):
    인증키 = "rx1J4yPTJLMypgEsmw%2BuyunTeZzwwUbRPrwPUkkYNxyUDpxomPQu1Hjz8vXE%2BzLfrzvFQdKPXqp%2FvonN%2Bc77iA%3D%3D"
    url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey="
    url = url + 인증키
    url = url + "&pageNo=1&dataCd=ASOS&dateCd=DAY&dataType=JSON"
    url = url + "&numOfRows=" + str(행갯수)
    url = url + "&startDt=" + str(시작일) + "&endDt=" + str(종료일)
    url = url + "&stnIds=" + str(지점코드)

#     print(url)
    
    # url을 이용해 데이터 요청
    soup_json = requests.get(url)
    if soup_json.status_code != 200:
        exit("데이터를 받아오지 못함")
    json_obj = json.loads(soup_json.text)
    
    return json_obj

In [13]:
import json

soup = urljson(20220101, 20220131, 108)
numRows = soup["response"]["body"]["totalCount"]  # 전체 조회 데이터 개수 확인
soup_obj = urljson(20220101, 20220131, 108, numRows)  # 전체 데이터 개수를 이용한 조회
items = soup_obj["response"]["body"]["items"]['item'] # 조회된 데이터에서 item 만 추출/ 리스트 형식으로 출력


df1 = pd.DataFrame(items)
df1 = df1[['stnNm', "tm", "avgTa", "minTa",'maxTa','sumRn']]
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   stnNm   31 non-null     object
 1   tm      31 non-null     object
 2   avgTa   31 non-null     object
 3   minTa   31 non-null     object
 4   maxTa   31 non-null     object
 5   sumRn   31 non-null     object
dtypes: object(6)
memory usage: 1.6+ KB


In [14]:
df1.head()

,stnNm,tm,avgTa,minTa,maxTa,sumRn
0,서울,2022-01-01,-4.3,-10.2,2.3,
1,서울,2022-01-02,-1.3,-5.2,3.0,0.3
2,서울,2022-01-03,-1.9,-8.0,2.5,0.0
3,서울,2022-01-04,-2.5,-5.6,1.0,0.0
4,서울,2022-01-05,-2.8,-7.8,1.9,
